In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path("..").resolve()))

# Explanation of config

## Section

Each section defines how we construct each component in the training. Most corresponds to a PyTorch component (e.g. *model* for `torch.nn.Module`, *data.dataset* for `torch.utils.data.Dataset`). Here are the custom components in this project:

In [2]:
from src.pixseg.pipeline import TensorboardLogger, Trainer, WandbLogger
from src.pixseg.utils.transform import SegmentationAugment

print("You may check their __init__ method to see how they are constructed")
print("[data.augment]", SegmentationAugment.__name__, ":", SegmentationAugment.__doc__)
print("[trainer]", Trainer.__name__, ":", Trainer.__doc__.splitlines()[0])  # type: ignore
print("[paths] don't have associated class")
print("[log.wandb]", WandbLogger.__name__, ":", WandbLogger.__doc__)
print("[log.tensorboard]", TensorboardLogger.__name__, ":", TensorboardLogger.__doc__)

You may check their __init__ method to see how they are constructed
[data.augment] SegmentationAugment : Default data augmentations for semantic segmentation
[trainer] Trainer : Repeatedly run training and validation loop, provide detailed logs
[paths] don't have associated class
[log.wandb] WandbLogger : Save log results and optionally snapshots to wandb
[log.tensorboard] TensorboardLogger : Save log results and snapshots to Tensorboard


## Fields
Most section have the field *"params"*. This will be sent as kwargs to the component.

### Constructor field
Some sections allow user to choose the constructor. Here is the list of such sections and the builtin options (you may register more)

In [3]:
from pprint import pprint

from src.pixseg.datasets import DATASET_ZOO
from src.pixseg.learn import (
    CLASS_WEIGHTINGS,
    CRITERION_ZOO,
    LR_SCHEDULER_ZOO,
    OPTIMIZER_ZOO,
)
from src.pixseg.models import MODEL_ZOO

# fmt: off
zoos: dict[str, dict] = {
    "[model.model]": MODEL_ZOO, "[data.dataset.dataset]": DATASET_ZOO, 
    "[criterion.criterion]": CRITERION_ZOO, "[criterion.class_weight]": CLASS_WEIGHTINGS, 
    "[optimizer.optimizer]": OPTIMIZER_ZOO, "[lr_scheduler.lr_scheduler]": LR_SCHEDULER_ZOO,
}
# fmt: on

for k, v in zoos.items():
    print(f"\n{k} {len(v)} items")
    pprint(sorted(v.keys()), width=100, compact=True)


[model.model] 20 items
['bisenet_resnet18', 'bisenet_resnet50', 'bisenet_xception', 'deeplabv3_mobilenet_v3_large',
 'deeplabv3_resnet101', 'deeplabv3_resnet18', 'deeplabv3_resnet50', 'enet', 'fcn_resnet101',
 'fcn_resnet50', 'fcn_vgg16', 'lraspp_mobilenet_v3_large', 'lraspp_resnet18', 'pspnet_resnet50',
 'sfnet_lite_resnet101', 'sfnet_lite_resnet18', 'sfnet_resnet101', 'sfnet_resnet18',
 'upernet_resnet101', 'upernet_resnet18']

[data.dataset.dataset] 11 items
['ADE20K', 'BDD100K', 'Cityscapes', 'Cityscapes{category}', 'Cityscapes{full}', 'LIP', 'Mapillary',
 'SBD', 'VOC', 'coco', 'coco{voc}']

[criterion.criterion] 3 items
['CrossEntropyLoss', 'DiceLoss', 'FocalLoss']

[criterion.class_weight] 5 items
['class_frequency', 'effective_number', 'log_frequency', 'none', 'sqrt_frequency']

[optimizer.optimizer] 3 items
['Adam', 'Padam', 'SGD']

[lr_scheduler.lr_scheduler] 4 items
['CosineAnnealingLR', 'OneCycleLR', 'PolynomialLR', 'StepLR']


### Special fields

`[model.state_file]` path to model's state dict  
`[data.dataset.pad_crop_size]` The size to pad or crop each image  
`[data.loader.num_workers]` applies to both train and eval dataloader  
`[criterion.class_weight]` weight given to each class in weighted loss. See above for valid name  
`[criterion.aux_weight]` alpha for auxiliary weight. You may want to set *aux_loss=True* in *[model.params]* for this to take effect  
`[optimizer.effective_batch_size]` the actual optimization batch size. Must be a multiple of *[data.loader.params.batch_size]*  
`[trainer.device]` if *"auto"*, choose cpu or cuda automatically  
`[paths.runs_folder]` folder to store logs, checkpoints and snapshots locally

## Note

`[data.dataset.pad_crop_size], [data.loader.params], [data.augment.params]` are only applied to the training dataset/loader. Evaluations are always done one by one with default augmentations. To use different test-time strategies, please see `tasks\evaluation.ipynb` or `tasks\inference.ipynb`